<a href="https://colab.research.google.com/github/RickyDoan/DL-Tensor-Flow-Churn-Prediction/blob/main/ANN_Churn_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("blastchar/telco-customer-churn")

print("Path to dataset files:", path)

In [ ]:
path

In [ ]:
import os

filepath = os.path.join(path, "WA_Fn-UseC_-Telco-Customer-Churn.csv")
df = pd.read_csv(filepath)
print(df.shape)
df.head()

# Cleaning Data

In [ ]:
# Drop unimportant columns

df.drop('customerID', axis=1, inplace=True)

In [ ]:
# Check null values
df.isnull().sum()

In [ ]:
# Check duplicate values
df.duplicated().sum()

In [ ]:
df.drop_duplicates(inplace=True, keep='first')

In [ ]:
df.duplicated().sum()

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df[df['TotalCharges']==" "]

In [ ]:
df[df['TotalCharges']==" "].shape

In [ ]:
df = df[df['TotalCharges']!=" "]

In [ ]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'])


In [ ]:
df['TotalCharges'].dtype

In [ ]:
df.shape

In [ ]:
sns.histplot(df, x = 'Churn', hue='Churn')

In [ ]:
cm_yes_churn_tennure = df[df['Churn']=='Yes']['tenure']
cm_no_churn_tennure = df[df['Churn']=='No']['tenure']

# blood_sugar_men = [113, 85, 90, 150, 149, 88, 93, 115, 135, 80, 77, 82, 129]
# blood_sugar_women = [67, 98, 89, 120, 133, 150, 84, 69, 89, 79, 120, 112, 100]
plt.hist([cm_yes_churn_tennure, cm_no_churn_tennure], color=['red', 'green'], label=['Yes', 'No'])
plt.legend()
plt.xlabel('Tenure')
plt.ylabel('Count')
plt.title('Churn vs Tenure')
plt.show()


In [ ]:
churn_yes_MonthlyCharges = df[df['Churn']=='Yes']['MonthlyCharges']
churn_no_MonthlyCharges = df[df['Churn']=='No']['MonthlyCharges']
plt.hist([churn_yes_MonthlyCharges, churn_no_MonthlyCharges], color=['red', 'green'], label=['Yes', 'No'])
plt.legend()
plt.xlabel('MonthlyCharges')
plt.ylabel('Count')
plt.title('Churn vs MonthlyCharges')
plt.show()

# Check Spelling

In [ ]:
for col in df.select_dtypes(include=('object')):
    print(f'{col}: {df[col].unique()}')

In [ ]:
df['MultipleLines'].replace('No phone service', 'No', inplace=True)
df['OnlineSecurity'].replace('No internet service', 'No', inplace=True)
df['OnlineBackup'].replace('No internet service', 'No', inplace=True)
df['DeviceProtection'].replace('No internet service', 'No', inplace=True)
df['TechSupport'].replace('No internet service', 'No', inplace=True)
df['StreamingTV'].replace('No internet service', 'No', inplace=True)
df['StreamingMovies'].replace('No internet service', 'No', inplace=True)


In [ ]:
for col in df.select_dtypes(include=('object')):
    print(f'{col}: {df[col].unique()}')

In [ ]:
len(df['gender'].unique())

In [ ]:
holder_column = []
for value in df.select_dtypes(include=('object')):
    if len(df[value].unique()) == 2:
        holder_column.append(value)

holder_column


In [ ]:
list_encoder = ['Partner',
 'Dependents',
 'PhoneService',
 'MultipleLines',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'PaperlessBilling',
 'Churn']

In [ ]:
for col in list_encoder:
    df[col] = df[col].map({'Yes': 1, 'No': 0})

In [ ]:
for col in df.select_dtypes(include=('object')):
    print(f'{col}: {df[col].unique()}')

In [ ]:
list_categories = ['gender', 'InternetService', 'Contract', 'PaymentMethod']

df = pd.get_dummies( df, columns =list_categories, drop_first=True, dtype=int)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
for col in df.columns:
    print(f'{col}: {df[col].unique()}')

In [ ]:
list_numerical = ['tenure', 'MonthlyCharges', 'TotalCharges']
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[list_numerical] = scaler.fit_transform(df[list_numerical])

In [ ]:
for col in df.columns:
    print(f'{col}: {df[col].unique()}')

In [ ]:
# Check VIF or Correlation

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def check_vif(data):
    df = pd.DataFrame()
    df["features"] = data.columns
    df["VIF"] = [variance_inflation_factor(data.values, i) for i in range(data.shape[1])]
    return df

df_vif = check_vif(df.drop('Churn', axis=1))
df_vif

In [ ]:
df_vif[df_vif['VIF']>20].sort_values(by='VIF', ascending=False).features.tolist()

In [ ]:
df.drop(df_vif[df_vif['VIF']>20].sort_values(by='VIF', ascending=False).features.tolist(), axis=1, inplace=True)


In [ ]:
df_vif = check_vif(df.drop('Churn', axis=1))
df_vif

In [ ]:
df_vif[df_vif['VIF']>10].sort_values(by='VIF', ascending=False).features.tolist()

In [ ]:
df.drop(df_vif[df_vif['VIF']>10].sort_values(by='VIF', ascending=False).features.tolist(), axis=1, inplace=True)

In [ ]:
df_vif = check_vif(df.drop('Churn', axis=1))
df_vif

In [ ]:
print(df.shape)
df.head()

# Training model

In [ ]:
X = df.drop('Churn', axis=1)
y = df['Churn']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model  = keras.Sequential([
    keras.layers.Dense(128, input_shape=(17,), activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
with tf.device('/GPU:0'):
    model.fit(X_train, y_train, epochs=100)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
y_pred = model.predict(X_test)
y_pred[:5]

In [ ]:
y_predict = []
for i in range(len(y_pred)):
    if y_pred[i] >= 0.5:
       y_predict.append(1)
    else:
        y_predict.append(0)

In [ ]:
y_predict[:10]

In [ ]:
y_test[:10].values

In [ ]:
report = classification_report(y_test, y_predict)
print(report)

In [ ]:
cm  = confusion_matrix(y_test, y_predict)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')